# Practice Notebook

This notebook will give you a chance to use some of the things that you've learned about in the workshop so far. In particular, this notebook will focus on using the packages we've discussed in some applications you might come across in your own work.

## Exercise 1: AR(1) Example

In this exercise, you should finish developing the code necessary to simulate an AR(1) process. We have filled some of the code in for you to give you a start.

The AR(1) process will take the following form

\begin{align*}
  x_{t+1} &= p x_t + \varepsilon_{t+1} \\
  \varepsilon_{t+1} &\sim_{i.i.d.} \phi
\end{align*}

Notice that we are not requiring that $\phi$ be a Normal distribution -- Our code should allow any distribution to fit into the process

In [ ]:
using Distributions
using PlotlyJS

In [ ]:
"""
We create a type that holds the two relevant elements of the autoregressive
process: the coefficient and the distribution. We will use this type to create
code that simulates the process
"""
type AR1
    p::Float64
    d::Distributions.Distribution
end

"""
This function performs a single step of the autoregressive process by taking the
process and its current value and returning the value for tomorrow
"""
one_step(ar1::AR1, x::Float64) = ar1.p*x + rand(ar1.d)


function simulate(ar1::AR1, T=25, x0=0.0)
    # Unpack the coefficient and distribution
    p, d = ar1.p, ar1.d

    # Allocate space that we will fill during simulation
    # and set first element to initial condition
    x = Array(Float64, T)
    x[1] = x0

    # TODO: write code to simulate

    # Return the simulated values
    return x
end

Now using your new simulate function, generate two time series with the same autocorrelation coefficient (0.90 for example) but different shock distributions. If you can't decide what distributions to use, try `Normal(0.0, 0.58)` and `Uniform(-1.0, 1.0)`. Plot these two time series.

In [ ]:
# Insert code here

## Exercise 2: Evaluating Likelihoods

Using the code that you have written above, write a function that takes a coefficient parameter and distribution and evaluates the likelihood of a simulated path of your process -- Make sure to take advantage of the functions (like `pdf`) that `Distributions.jl` exposes for you.

In [ ]:
# Insert code below

## Exercise 3: Maximum Likelihood

Now that you can evaluate likelihood functions, you should be able to perform maximum likelihood. Choose a particular distribution and maximize over the autocorrelation coefficient and the parameters that describe the distribution

In [1]:
# Insert code below

## Exercise 4: Cake eating problem

In this problem, we will solve the canonical finite horizon "cake eating problem" as an example of dynamic programming. In particular, consider an agent who has a cake of size $X$. The agent lives for $T$ periods, discounts the future at rate $\beta$, and values period by period consumption according to $\sqrt(c)$.

You could write his dynamic programming problem by

\begin{align*}
  V_t(x) &= \max_{c} \sqrt(c) + \beta V_{t+1}(x - c)
\end{align*}

with a terminal condition of

\begin{align*}
  V_{T+1}(x) = 0
\end{align*}

In [ ]:
using Interpolations
using Optim

In [ ]:
type CakeProblem
    # Parameters of cake eating problem
    β::Float64
    X::Float64
    T::Int

    # Grid to solve problem on
    xgrid::LinSpace

    # Value and policy grid
    # (2 dimensions -- x down columsn and T across rows)
    vgrid::Array{Float64, 2}
    cgrid::Array{Float64, 2}
end

function CakeProblem(β=0.95, X=1.0, T=3, xmin=0.0, xmax=X, nx=250)
    # Allocate space for grids
    vgrid = Array(Float64, nx, T)
    cgrid = Array(Float64, nx, T)

    # Build grid of x values we will have in each period
    xgrid = linspace(xmin, xmax, nx)

    return CakeProblem(β, X, T, xgrid, vgrid, cgrid)
end

In [ ]:
"""
This function generates a solution to the cake problem that is described above. It
stores the solutions inside of the arrays that were allocated inside the CakeProblem
type.
"""
function solve!(cp::CakeProblem)
    # Pull out parameters of cake problem
    β, X, T = cp.β, cp.X, cp.T

    # At period T, we will eat all remaining cake
    cp.cgrid[:, T] = cp.xgrid
    cp.vgrid[:, T] = sqrt(cp.cgrid[:, T])

    # Use backwards recursion to get policy and value function
    for t=T-1:-1:1
        println(t)
        # TODO: Create current value interpolator (use linear)

        # Iterate over all values of x
        for (ix, x) in enumerate(cp.xgrid)
            # If we have no good left then we can't eat anything
            if x < 1e-15
                cstar = 0.0
            else
                # TODO: Write a short function that we can minimize to get optimal value

                # TODO: Formulate optimization problem to get policy

            end

            # Fill policy and value today
            cp.cgrid[ix, t] = 0.0  # TODO: Fill this with correct value
            cp.vgrid[ix, t] = 0.0  # TODO: Fill this with correct value
        end
    end

    return nothing
end

Using your solution, plot the policy function for each period